In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pickle
import os
import sys
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [ ]:
PACKAGE_DIR = os.path.abspath('../Fork/striatum')
DATA_DIR = os.path.abspath('../../Data/yahoo!_front_page_today_module_user_click_log/R6')
DATA_FILE_NAMES = ['ydata-fp-td-clicks-v1_0.2009050{}.gz'.format(str(d + 3)) for d in range(7)]
MODEL_DIR = os.path.abspath('models')

In [ ]:
sys.path.insert(0, PACKAGE_DIR)
from striatum.bandit import hyblinucb
from utils.data import *
from utils.policy import *

In [ ]:
data_paths = []
for data_file in DATA_FILE_NAMES:
    data_paths.append(os.path.join(DATA_DIR, data_file))

In [ ]:
policy_learner = PolicyLearner(hyblinucb.HybLinUCB, shared_context_dimension=35, unshared_context_dimension=6, 
                               is_shared_context_first=True, alpha=0.4, seed=0, reset_freq=1000000)
param_grid = {'data_size': [0.01, 0.05, 0.1, 0.2, 0.3, 1]}

In [ ]:
res = search_param(policy_learner, param_grid, parse_data, data_paths=data_paths, share_context=True)

In [ ]:
if not os.path.isdir(MODEL_DIR):
    os.mkdir(MODEL_DIR)

In [ ]:
with open(os.path.join(MODEL_DIR, 'model_hybrid_linucb.pkl'), 'wb') as file:
    pickle.dump(res, file)

In [ ]:
with open(os.path.join(MODEL_DIR, 'model_hybrid_linucb.pkl'), 'rb') as file:
    res = pickle.load(file)

In [ ]:
best_param = res['param'][np.argmax(res['avg_reward'])]
cum_ctr = np.max(res['avg_reward'])
print('Best Parameter: {0}\nCumulative CTR: {1:0.4%}'.format(best_param, cum_ctr))
fig, ax = plt.subplots(figsize=(10, 6))
_ = ax.plot(res['seq_avg_reward'][np.argmax(res['avg_reward'])], color='mediumblue', 
            label='LinUCB with Hybrid Linear Models', linewidth=1)
ax.axhline(y=cum_ctr, color='black', linestyle='--', linewidth=1)
ax.set_xlabel('Num. of Events')
ax.set_ylabel('CTR')
ax.set_ylim([0.0, 0.1])
ax.legend(loc='best')
_ = ax.set_title('Average Reward', fontsize=12, fontweight='bold')